In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp auto

In [ ]:
#| export
from collections import defaultdict
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import numpy as np
import optuna
import pandas as pd
from sklearn.base import BaseEstimator, clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from utilsforecast.compat import DataFrame
from utilsforecast.losses import mase, smape
from utilsforecast.processing import counts_by_id
from utilsforecast.validation import validate_freq

from mlforecast import MLForecast
from mlforecast.core import Freq, Models, _get_model_name, _name_models
from mlforecast.lag_transforms import ExpandingMean, ExponentiallyWeightedMean, RollingMean
from mlforecast.optimization import mlforecast_objective
from mlforecast.target_transforms import Differences, LocalStandardScaler, GlobalSklearnTransformer

In [ ]:
#| export
def lightgbm_space(trial):
    return {
        "bagging_freq": 1,
        "learning_rate": 0.05,
        "verbosity": -1,        
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 4096, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        "objective": trial.suggest_categorical("objective", ['l1', 'l2']),
    }

def xgboost_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'bagging_freq': trial.suggest_float('bagging_freq', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'min_data_in_leaf': trial.suggest_float('min_data_in_leaf', 1, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
    }
    
def catboost_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        'depth': trial.suggest_int('depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'min_data_in_leaf': trial.suggest_float('min_data_in_leaf', 1, 100),
    }
    
def linear_regression_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }
    
def ridge_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0)
    }
    
def lasso_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0)
    }
    
def elasticnet_space(trial):
    return {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        'alpha': trial.suggest_float('alpha', 0.001, 10.0),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0)
    }

def random_forest_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_child_samples', 1, 100),
        'max_features': trial.suggest_float('max_features', 0.5, 1.0),
        "criterion": trial.suggest_categorical("criterion", ['squared_error', 'poisson']),
    }

In [ ]:
#| exporti
_default_model_spaces = {
    'LGBMRegressor': lightgbm_space,
    'XGBRegressor': xgboost_space,
    'CatBoostRegressor': catboost_space,
    'LinearRegression': linear_regression_space,
    'Ridge': ridge_space,
    'Lasso': lasso_space,
    'ElasticNet': elasticnet_space,
    'RandomForest': random_forest_space,
}

_ModelWithConfig = Tuple[BaseEstimator, Optional[Dict[str, Callable]]]

In [ ]:
#| export
class AutoMLForecast:

    def __init__(
        self,
        models_with_configs: Union[List[_ModelWithConfig], Dict[str, _ModelWithConfig]],
        freq: Freq,
        season_length: int,
        init_config: Optional[Callable] = None,
        fit_config: Optional[Callable] = None,
        num_threads: int = 1,
    ):
        self.freq = freq
        self.season_length = season_length
        self.num_threads = num_threads
        if isinstance(models_with_configs, list):
            names = _name_models([_get_model_name(m) for m, _ in models_with_configs])
            self.models_with_configs = {
                name: (model, config)
                for name, (model, config) in zip(names, models_with_configs)
            }
        elif isinstance(models_with_configs, dict):
            self.models_with_configs = models_with_configs
        else:
            raise ValueError(
                '`models_with_configs` should be a list of tuples '
                'or a dict from str to tuple.'
            )
        for name, (model, config) in self.models_with_configs.items():
            if config is not None:
                continue
            if name not in _default_model_spaces:
                raise NotImplementedError(
                    f"{name} does not have a default config. Please provide one."
                )
            self.models_with_configs[name] = (model, _default_model_spaces[name])
        self.init_config = init_config
        if fit_config is not None:
            self.fit_config = fit_config
        else:
            self.fit_config = lambda trial: {}

    def _seasonality_based_config(
        self,
        h: int,
        min_samples: int,
        min_value: float,
    ) -> Callable:
        # target transforms  
        candidate_targ_tfms = [
            None,
            [LocalStandardScaler()],
            [Differences([1]), LocalStandardScaler()],
        ]
        log1p_tfm = GlobalSklearnTransformer(
            FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
        )
        if min_value >= 0:
            candidate_targ_tfms.extend(
                [
                    [log1p_tfm, LocalStandardScaler()],
                    [log1p_tfm, Differences([1]), LocalStandardScaler()],
                ]
            )
        # we leave two seasonal periods for the features and model
        if self.season_length > 1 and min_samples > 3 * self.season_length + 1:
            candidate_targ_tfms.append([Differences([1, self.season_length]), LocalStandardScaler()])
            if min_value >= 0:
                candidate_targ_tfms.append(
                    [log1p_tfm, Differences([1, self.season_length]), LocalStandardScaler()],
                )

        # lags
        candidate_lags = [None]
        if self.season_length > 1:
            candidate_lags.append([self.season_length])
        seasonality2extra_candidate_lags = {
            7: [
                [7, 14],
                [7, 28],
            ],
            12: [range(1, 13)],
            24: [
                range(1, 25),
                range(24, 24 * 7 + 1, 24),
            ],
        }
        candidate_lags.extend(
            seasonality2extra_candidate_lags.get(self.season_length, [])
        )
        if h >= 2 * self.season_length:
            candidate_lags.extend(
                [
                    range(self.season_length, h + 1, self.season_length),
                    [h],
                    [self.season_length, h],
                ]
            )

        # lag transforms
        candidate_lag_tfms = [None, {1: [ExponentiallyWeightedMean(0.9)]}]
        if self.season_length > 1:
            candidate_lag_tfms.append(
                {
                    1: [ExponentiallyWeightedMean(0.9)],
                    self.season_length: [
                        RollingMean(window_size=self.season_length, min_samples=1),
                    ]
                }
            )
        if self.season_length != h:
            candidate_lag_tfms.append(
                {
                    1: [ExponentiallyWeightedMean(0.9)],
                    self.season_length: [
                        RollingMean(window_size=self.season_length, min_samples=1),
                    ],
                    h: [
                        RollingMean(window_size=self.season_length, min_samples=1),
                    ]
                }
            )

        # date features
        seasonality2date_features = {
            1: ['year'],
            4: ['quarter', 'year'],
            7: ['weekday', 'month', 'year'],
            12: ['month', 'year'],
            24: ['hour', 'weekday', 'month', 'year'],
            52: ['week', 'year'],
            60: ['weekday', 'hour', 'second'],
        }
        date_features = seasonality2date_features.get(self.season_length, None)
        if isinstance(self.freq, int):
            date_features = None
        if date_features is not None:
            use_date_features = trial.suggest_int('use_date_features', 0, 1)
            if not use_date_features:
                date_features = None        

        def config(trial):
            # target transforms
            targ_tfms_idx = trial.suggest_categorical(
                'target_transforms_idx', range(len(candidate_targ_tfms))
            )
            target_transforms = candidate_targ_tfms[targ_tfms_idx]
    
            # lags
            lags_idx = trial.suggest_categorical('lags_idx', range(len(candidate_lags)))
            lags = candidate_lags[lags_idx]
    
            # lag transforms
            if candidate_lag_tfms:
                lag_tfms_idx = trial.suggest_categorical(
                    'lag_transforms_idx', range(len(candidate_lag_tfms))
                )
                lag_transforms = candidate_lag_tfms[lag_tfms_idx]
            else:
                lag_transforms = None
            
            return {
                'lags': lags,
                'target_transforms': target_transforms,
                'lag_transforms': lag_transforms,
                'date_features': date_features,            
            }

        return config

    def fit(
        self,
        df: DataFrame,
        n_windows: int,
        h: int,
        num_samples: int,        
        loss: Optional[Callable] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        study_kwargs: Optional[Dict[str, Any]] = None,
        optimize_kwargs: Optional[Dict[str, Any]] = None,
    ) -> 'AutoMLForecast':
        validate_freq(df[time_col], self.freq)
        if self.init_config is not None:
            init_config = self.init_config
        else:
            min_size = counts_by_id(df, id_col)['counts'].min()
            min_train_size = min_size - n_windows * h
            init_config = self._seasonality_based_config(
                h=h,
                min_samples=min_train_size,
                min_value=df[target_col].min(),
            )

        if loss is None:
            def loss(df, train_df):
                return smape(df, models=['model'])['model'].mean()
        if study_kwargs is None:
            study_kwargs = {}
        if 'sampler' not in study_kwargs:
            # for reproducibility
            study_kwargs['sampler'] = optuna.samplers.TPESampler(seed=0)
        if optimize_kwargs is None:
            optimize_kwargs = {}
        if 'n_jobs' not in optimize_kwargs:
            optimize_kwargs['n_jobs'] = 1

        self.results_ = []
        self.models_ = {}
        for name, (model, model_config) in self.models_with_configs.items():
            def config_fn(trial: optuna.Trial) -> float:
                return {
                    'model_params': model_config(trial),
                    'mlf_init_params': {
                        **init_config(trial),
                        'num_threads': self.num_threads,
                    },
                    'mlf_fit_params': self.fit_config(trial)
                }

            objective = mlforecast_objective(
                df=df,
                config_fn=config_fn,
                eval_fn=loss,
                model=model,
                freq=self.freq,
                n_windows=n_windows,
                h=h,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
            )
            study = optuna.create_study(direction='minimize', **study_kwargs)
            study.optimize(objective, n_trials=num_samples, **optimize_kwargs)
            self.results_.append(study)
            best_config = study.best_trial.user_attrs['config']            
            best_model = clone(model)
            best_model.set_params(**best_config['model_params'])
            self.models_[name] = MLForecast(
                models={name: best_model},
                freq=self.freq,
                **best_config['mlf_init_params']
            )
            self.models_[name].fit(
                df,
                **best_config['mlf_fit_params']
            )
        return self

    def predict(
        self,
        h: int,
        X_df: Optional[DataFrame] = None,
    ) -> DataFrame:
        all_preds = None
        for name, model in self.models_.items():
            preds = model.predict(h=h, X_df=X_df)
            if all_preds is None:
                all_preds = preds
            else:
                all_preds[name] = preds[name]
        return all_preds

    def save(self, path: str) -> None:
        for name, model in self.models_.items():
            model.save(f'{path}/{name}')

In [ ]:
import warnings

import lightgbm as lgb
from datasetsforecast.m4 import M4, M4Evaluation, M4Info
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def train_valid_split(group):
    df, *_ = M4.load(directory='data', group=group)
    df['ds'] = df['ds'].astype('int')
    horizon = M4Info[group].horizon
    valid = df.groupby('unique_id').tail(horizon).copy()
    train = df.drop(valid.index).reset_index(drop=True)
    return train, valid

In [ ]:
import pickle

In [ ]:
def run_group(group):
    train, valid = train_valid_split(group)
    train['unique_id'] = train['unique_id'].astype('category')
    valid['unique_id'] = valid['unique_id'].astype(train['unique_id'].dtype)
    info = M4Info[group]
    h = info.horizon
    season_length = info.seasonality
    ridge_pipeline = make_pipeline(
        ColumnTransformer(
            [('encoder', OneHotEncoder(), ['unique_id'])],
            remainder='passthrough',
        ),
        Ridge(),
    )
    auto_mlf = AutoMLForecast(
        freq=1,
        season_length=season_length,
        models_with_configs=[(lgb.LGBMRegressor(), None)],
        fit_config=lambda trial: {'static_features': ['unique_id']} if trial.suggest_int('static_features', 0, 1) else {},
    )
    with warnings.catch_warnings(record=False):
        warnings.simplefilter('ignore', category=UserWarning)
        auto_mlf.fit(
            df=train,
            n_windows=2,
            h=h,
            num_samples=30,
            optimize_kwargs={'timeout': 60 * 60},
        )
    with open(f'{group}_opt.pkl', 'wb') as f:
        pickle.dump(auto_mlf.results_, f)
    preds = auto_mlf.predict(h=h)
    for model in auto_mlf.models_with_configs.keys():
        print(model)
        print(M4Evaluation.evaluate('data', group, preds[model].values.reshape(-1, h)))    

In [ ]:
import time

optuna.logging.set_verbosity(optuna.logging.ERROR)

for group in ('Yearly', 'Quarterly', 'Weekly', 'Hourly', 'Monthly'):
    print(f'Running {group}')
    start = time.perf_counter()
    run_group(group)
    print(f'{group} took {(time.perf_counter() - start) / 60:.1f} minutes')